In [54]:
import numpy as np
import os
import tensorflow as tf

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

###### Do not modify here ###### 

Inputs = np.concatenate((X_train1, X_valid1), axis=0)
Labels = np.concatenate((y_train1, y_valid1), axis=0)

print(X_train1.shape)
print(X_valid1.shape)
print(X_test1.shape)
print(Inputs.shape)
print(Labels.shape)

# one_hot encoding 0, 1, 2, 3, 4 for each set

n_train, k = X_train1.shape
n_valid, k = X_valid1.shape
n_test, k = X_test1.shape

tmp_y = np.zeros([n_train, 5])
for i in range(n_train):
    tmp_y[i][y_train1[i]] = 1
y_train1 = tmp_y

tmp_y = np.zeros([n_valid, 5])
for i in range(n_valid):
    tmp_y[i][y_valid1[i]] = 1
y_valid1 = tmp_y

tmp_y = np.zeros([n_test, 5])
for i in range(n_test):
    tmp_y[i][y_test1[i]] = 1
y_test1 = tmp_y


# adding layer function
def add_layer(name, inputs, in_size, out_size, Weights_initializer, activation, dropout):
    Weights = tf.get_variable("Weights"+name, initializer=Weights_initializer, shape=[in_size, out_size], dtype=tf.float32)
    biases = tf.get_variable("biases"+name, initializer=tf.random_uniform_initializer(-1.0, 1.0, dtype=tf.float32), shape=[1, out_size], dtype=tf.float32)
    XW_plus_b = tf.matmul(inputs, Weights) + biases
    # add dropout
    XW_plus_b = tf.nn.dropout(XW_plus_b, (1-dropout))
    if activation is None:
        outputs = XW_plus_b
    else:
        outputs = activation(XW_plus_b)
    return outputs

# computing accuravy function
def compute_accuracy(Valid_X, Valid_y):
    global outputs
    y_pre = sess.run(outputs, feed_dict={x: Valid_X, y: Valid_y})
    correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(Valid_y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={x: Valid_X, y: Valid_y})
    return result
    
    
# define placeholder for inputs
x = tf.placeholder(tf.float32, [None, k])
y = tf.placeholder(tf.float32, [None, 5])

# add 5 hidden layers with 128 neurals each
h1 = add_layer("h1", x, k, 128, tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode="FAN_IN", uniform=True), tf.nn.elu, 0.3)
h2 = add_layer("h2", h1, 128, 128, tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode="FAN_IN", uniform=True), tf.nn.elu, 0.3)
h3 = add_layer("h3", h2, 128, 128, tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode="FAN_IN", uniform=True), tf.nn.elu, 0.3)
h4 = add_layer("h4", h3, 128, 128, tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode="FAN_IN", uniform=True), tf.nn.elu, 0.3)
h5 = add_layer("h5", h4, 128, 128, tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode="FAN_IN", uniform=True), tf.nn.elu, 0.3)

# add outputs softmax layer with 5 neurals
outputs = add_layer("outputs", h5, 128, 5, tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode="FAN_IN", uniform=True), tf.nn.softmax, 0.0)

# cross entropy loss function
cross_entropy = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=y))

# training iteration and define Adam optimizer with learning rate 0.01
train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)

#start TF session
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(1000):
    sess.run(train_step, feed_dict={x: X_train1, y: y_train1})
    if i%50 == 0:
        print(compute_accuracy(X_valid1, y_valid1))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
(28038, 784)
(2558, 784)
(5139, 784)
(30596, 784)
(30596,)
0.24355
0.969507
0.978108
0.981626
0.981626
0.981235
0.982799
0.984754
0.984754
0.981626
0.986317
0.984363
0.984754
0.985145
0.983972
0.982408
0.98319
0.985927
0.985536
0.983972
